# Kronecker Theta Functions and Kronecker Double Series

This notebook aims to setup code for exploring Kronecker Theta Functions and Kronecker Double Series. The aims are to:

- give defintitions of the Kronecker Theta function in terms of ratios of Jacobi Theta functions and Weierstrass Sigma functions.
- numerically verify that the functions can be correctly plotted and tested.
- explore the series expansions in some dynamic systems with known sigma function ratio solutions and see if they yield new insights.
- ultimately demonstrate the importance of the Kronecker theta function in nonlinear coupled ordinary differential equations and associated applications.

**Important References**

The term *Kronecker theta function* for the ratio of Jacobi theta functions was first heard by me in 2023 when reading the 2016 publication 
[*Elliptic Functions According to Eisenstein and Kronecker: An Update*](https://webusers.imj-prg.fr/~pierre.charollois/Charollois-Sczech_EMS.pdf) by Pierre Charollois and Robert Sczech. They attribute the term to [*Algebraic theta functions and p-adic interpolation of Eisenstein-Kronecker numbers*]](https://arxiv.org/abs/math/0610163 by Kenichi Bannai and Shinichi Kobayashi published in 2006. It was the 1976 Andre Weil book that rekindled modern interest in this area of Kronecker's work and inspired the Charollois and Sczech update, and teh Weil book is based on some notes of Kroneckers lectures written up by F. von Dalwigk in 1891 that expanded on work by Eisenstein. Those notes were digitised and made available in German [here](https://webusers.imj-prg.fr/~pierre.charollois/Kronecker.html). It is also worth mentioning that in [*KRONECKER THETA FUNCTION AND A DECOMPOSITION THEOREM FOR THETA FUNCTIONS I*](https://arxiv.org/pdf/2012.01670.pdf) published in 2020 by ZHI-GUO LIU it was shown that meromorphic functions that satisfy certain conditions re poles and functional identities also satisfy a decomposition theorem in terms of Kronecker theta functions. Finally, it is to be noted that part of the work in Charollois and Sczech follows the 2006 paper [*Algebraic theta functions and p-adic interpolation of Eisenstein-Kronecker numbers*](https://arxiv.org/pdf/math/0610163.pdf) by Kenichi Bannai, Shinichi Kobayashi.

I was aware that the theta or sigma ratio could be expanded in a *Kronecker double series* and mentioned this in an appendix in the 2015 paper [*General complex envelope solutions of coupled-mode optics with quadratic or cubic nonlinearity*](https://arxiv.org/abs/1512.03092) and in that paper cited two sources for that claim one was *Elliptic Functions according to Eisenstein and Kronecker*, by Andre Weil, published in Springer-Verlag, 1972, and the other was *E. T. Whitaker and G. N. Watson, A Course of Modern Analysis, (Merchant Books, 1915)* (but I can't spot the reference to the Kronecker double series in the Whitaker and Watson book when I last checked, may be a specific edition is needed, although there are several useful points on double series and their convergence in general). I mentioned the double series in the Appendix as I thought a Fourier series could prove useful in optics but never got round to exploring it. That said, the form of the double series mentioned in my Appendix is not the same as the one in Charollois and Sczech, and it is theirs that has once again peaked my interest as it is a pole expansion that could prove useful in physical interpretations and perhaps even be used as an ansatz to find new solutions to dynamic systems that have so far proved illusive. This work will begin my exploration of the Kronecker theta function as a double series pole expansion.s)
4006

## Setting up the variables

In [75]:
from sympy import *
(xi, eta, u, tau, N, M, n, m, k, z, q, a, w, s) = symbols('''xi, eta, u, tau, N, M, n, m, k, z, q, a, w, s''')
Ser = Function('Ser')
pw = Function('pw') # Weierstrass P function
pwp = Function('pwp') # Derivative of Weierstrass P function
zw = Function('zw') # Weierstrass Zeta function
sigma = Function('sigma') # Weierstrass Sigma function
f = Function('f')
h = Function('h')
K = Function('K')
theta = Function('theta')
Theta = Function('Theta')
vartheta = IndexedBase('vartheta')
from math import prod

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [76]:
from numpy import linspace, absolute, angle, square, real, imag, conj, array as arraynp, vectorize, concatenate
import scipy.integrate
import matplotlib.pyplot as plt

# The package containing mpmath expressions for Weierstrass elliptic functions
from numerical_evaluation.weierstrass_modified import Weierstrass
we = Weierstrass()
from mpmath import exp as mpexp
from mpmath import jtheta, mpf
def mpc_to_float(mpc_val):
    return float(mpc_val.real) + float(mpc_val.imag)*1j

# %load_ext autoreload
# %autoreload 2

## The Kronecker theta function as a double series

### Kronecker double series definition
Starting from equation $(5)$ in Charollois and Sczech (in the limit that $N,M$ go to $\infty$)

In [77]:
Seq_dbl_series = Eq(Ser(xi, eta, u, tau), Sum(Sum(exp(2*I*pi*(-m*xi + n*eta))/(u + m + n*tau),(n,-N,N)),(m,-M,M)))
Seq_dbl_series

Eq(Ser(xi, eta, u, tau), Sum(exp(2*I*pi*(eta*n - m*xi))/(m + n*tau + u), (n, -N, N), (m, -M, M)))

### Jacobi theta function definition

Here the Jacobi theta function is defned and it will be denoted using $\theta$ as sympy is somewhat restricted by characters for functions (although vartheta works as a symbol). This theta function is intended to be the same Jacobi theta function used in Charollois and Sczech and it is commonly denoted $\mathcal{theta}_1$ elsewhere, e.g. https://mathworld.wolfram.com/JacobiThetaFunctions.html. It will be defined using the same series and products used in Charollois and Sczech. Furthermore, checks are performed to also confirm that the value coming from the $\mathrm{jtheta}(1,\pi z,\exp(i \pi \tau))$ function in the mpmath python package also agrees with the definition, which is important as that forms part of the numerical evaluation of Weierstrass elliptic functions used elsewhere.

In [78]:
Eq(theta(z,tau), vartheta[1])
tau_domain = im(tau)>0
q_tau = Eq(q, exp(2*I*pi*tau))
theta_sum_def = Eq(theta(z,tau), Sum(exp(2*I*pi*((n + Rational(1,2))**2*tau/2 + (n + Rational(1,2))*(z - Rational(1,2)))),(n,-N,N)))
theta_prod_def = Eq(theta(z,tau), 2*exp(I*pi*tau/4)*sin(pi*z)*Product((1-q**n)*(1-q**n*exp(2*I*pi*z))*(1-q**n*exp(-2*I*pi*z)),(n,1,N)))
tau_domain
q_tau
theta_sum_def
theta_prod_def

Eq(theta(z, tau), vartheta[1])

im(tau) > 0

Eq(q, exp(2*I*pi*tau))

Eq(theta(z, tau), Sum(exp(2*I*pi*(tau*(n + 1/2)**2/2 + (n + 1/2)*(z - 1/2))), (n, -N, N)))

Eq(theta(z, tau), 2*exp(I*pi*tau/4)*sin(pi*z)*Product((1 - q**n)*(-q**n*exp(-2*I*pi*z) + 1)*(-q**n*exp(2*I*pi*z) + 1), (n, 1, N)))

In [79]:
N_val = 200
z_val = 2.245
tau_val = 0.78+I*0.8
test_params = [(N,N_val),(z,z_val), (tau, tau_val)]
Eq(theta_sum_def.lhs.subs(test_params), theta_sum_def.rhs.subs(test_params).doit().evalf())
Eq(theta_prod_def.lhs.subs(test_params), theta_prod_def.rhs.subs(*q_tau.args).subs(test_params).doit().evalf())
Eq(theta(z_val, tau_val), mpc_to_float(jtheta(1,(z_val*pi).evalf(), exp(I*pi*tau_val).evalf())))

Eq(theta(2.245, 0.78 + 0.8*I), 0.603776148725747 + 0.43055767622146*I)

Eq(theta(2.245, 0.78 + 0.8*I), 0.603776148725689 + 0.430557676221541*I)

Eq(theta(2.245, 0.78 + 0.8*I), 0.603776148725747 + 0.43055767622146*I)

#### Derivatives of the Jacobi theta functions
The notation from mpmath python package to denote derivatives using a third paramter will be adopted here and the mpmath package will be used to evaluate them, that is:

$\theta(z,\tau,k)=\pi^k \mathrm{jtheta}(1,\pi z,\exp(i \pi \tau),k)$

and let:

In [80]:
Eq(theta(z,tau,k), Derivative(theta(z,tau),(z,k)))

Eq(theta(z, tau, k), Derivative(theta(z, tau), (z, k)))

### Kronecker's theorem relating the Kronecker theta function to the Kronecker double series
Kronecker's main result (which is Theorem 2.4 in Charollois and Sczech) is to express the ratio of Jacobi theta functions in terms of the double series as follows:

In [81]:
0<im(u)
im(u)<im(tau)
0<xi
xi<1

im(u) > 0

im(u) < im(tau)

xi > 0

xi < 1

In [82]:
Seq_dbl_ktheta = Eq(Ser(xi, eta, u, tau), exp(2*I*pi*xi*u)*theta(0,tau,1)*theta(u + eta + tau*xi, tau)/theta(u, tau)/theta(eta + tau*xi, tau))
Seq_dbl_ktheta

Eq(Ser(xi, eta, u, tau), theta(eta + tau*xi + u, tau)*theta(0, tau, 1)*exp(2*I*pi*u*xi)/(theta(u, tau)*theta(eta + tau*xi, tau)))

In [83]:
ktheta_dbl_series = Seq_dbl_series.subs(*Seq_dbl_ktheta.args)
ktheta_dbl_series

Eq(theta(eta + tau*xi + u, tau)*theta(0, tau, 1)*exp(2*I*pi*u*xi)/(theta(u, tau)*theta(eta + tau*xi, tau)), Sum(exp(2*I*pi*(eta*n - m*xi))/(m + n*tau + u), (n, -N, N), (m, -M, M)))

In [202]:
eta_val = 3.78
tau_val = 8.239765 + 2.3*I
u_val = 4.34144 + 0.8*I
xi_val = 0.435
N_val = 100
M_val = 100
S_dbl_subs_eta_tau_u_xi = [(eta, eta_val), (tau, tau_val), (u, u_val), (xi, xi_val), (N, N_val), (M, M_val)]

In [240]:
def theta_ratio_S(_xi, _eta, _u, _tau):
    result = (
        exp(2*I*pi*_u*_xi) * 
        jtheta(1,((_eta + _tau * _xi + _u) * pi).evalf(), exp(I*pi*_tau).evalf()) *
        pi * jtheta(1, 0, exp(I*pi*_tau).evalf(), 1) /
        (jtheta(1,(_u * pi).evalf(), exp(I*pi*_tau).evalf()) * jtheta(1,((_eta + _tau * _xi) * pi).evalf(), exp(I*pi*_tau).evalf()) )
    ).evalf()
    return result

def theta_ratio_S_u_diff(_xi, _eta, _u, _tau):
    result = (
        2*I*pi**2*_xi*exp(2*I*pi*_u*_xi) * 
        jtheta(1,((_eta + _tau * _xi + _u) * pi).evalf(), exp(I*pi*_tau).evalf()) *
        jtheta(1, 0, exp(I*pi*_tau).evalf(), 1) /
        (jtheta(1,(_u * pi).evalf(), exp(I*pi*_tau).evalf()) * jtheta(1,((_eta + _tau * _xi) * pi).evalf(), exp(I*pi*_tau).evalf()) ) +

        pi**2*exp(2*I*pi*_u*_xi) * jtheta(1, 0, exp(I*pi*_tau).evalf(), 1) * 
        jtheta(1,((_eta + _tau * _xi + _u) * pi).evalf(), exp(I*pi*_tau).evalf(), 1) /
        (jtheta(1,(_u * pi).evalf(), exp(I*pi*_tau).evalf()) * jtheta(1,((_eta + _tau * _xi) * pi).evalf(), exp(I*pi*_tau).evalf()) ) -

        pi**2*exp(2*I*pi*_u*_xi) * jtheta(1,((_eta + _tau * _xi + _u) * pi).evalf(), exp(I*pi*_tau).evalf()) *
        jtheta(1, 0, exp(I*pi*_tau).evalf(), 1) * jtheta(1,(_u * pi).evalf(), exp(I*pi*_tau).evalf(), 1) /
        (jtheta(1,(_u * pi).evalf(), exp(I*pi*_tau).evalf()) ** 2 * jtheta(1,((_eta + _tau * _xi) * pi).evalf(), exp(I*pi*_tau).evalf()) ) 

    ).evalf()
    return result

def dbl_series_S_summand(_xi, _eta, _u, _tau, _m, _n, _s):
    return (
        exp(2*I*pi*(- _m * _xi + _n * _eta))/(_u + _m + _n * _tau) ** _s
    ).evalf()

def dbl_series_S(_xi, _eta, _u, _tau, _M, _N, _s=1):
    result = 0
    for _m in range(-_M, _M + 1):
        for _n in range(-_N, _N + 1):
            result += (-1) ** (_s+1) * dbl_series_S_summand(_xi, _eta, _u, _tau, _m, _n, _s)
    return result

In [242]:
S_val_1 = Eq(Seq_dbl_ktheta.lhs, theta_ratio_S(xi_val, eta_val, u_val, tau_val))
S_val_2 = Eq(Seq_dbl_series.lhs, dbl_series_S(xi_val, eta_val, u_val, tau_val, M_val, N_val))
S_val_1
S_val_2
abs(S_val_1.rhs - S_val_2.rhs) / abs(S_val_1.rhs + S_val_2.rhs) / 2

Eq(Ser(xi, eta, u, tau), -0.463220020983809 - 0.567076331077644*I)

Eq(Ser(xi, eta, u, tau), -0.462091113701485 - 0.573383065191033*I)

0.00218128905779438

#### Checking the derivative also as it will converge faster

In [243]:
Eq(Derivative(ktheta_dbl_series.lhs,u), Derivative(ktheta_dbl_series.rhs,u))
Eq(diff(ktheta_dbl_series.lhs,u), diff(ktheta_dbl_series.rhs,u))

Eq(Derivative(theta(eta + tau*xi + u, tau)*theta(0, tau, 1)*exp(2*I*pi*u*xi)/(theta(u, tau)*theta(eta + tau*xi, tau)), u), Derivative(Sum(exp(2*I*pi*(eta*n - m*xi))/(m + n*tau + u), (n, -N, N), (m, -M, M)), u))

Eq(2*I*pi*xi*theta(eta + tau*xi + u, tau)*theta(0, tau, 1)*exp(2*I*pi*u*xi)/(theta(u, tau)*theta(eta + tau*xi, tau)) + theta(0, tau, 1)*exp(2*I*pi*u*xi)*Subs(Derivative(theta(_xi_1, tau), _xi_1), _xi_1, eta + tau*xi + u)/(theta(u, tau)*theta(eta + tau*xi, tau)) - theta(eta + tau*xi + u, tau)*theta(0, tau, 1)*exp(2*I*pi*u*xi)*Derivative(theta(u, tau), u)/(theta(u, tau)**2*theta(eta + tau*xi, tau)), Sum(-exp(2*I*pi*(eta*n - m*xi))/(m + n*tau + u)**2, (n, -N, N), (m, -M, M)))

In [239]:
Eq((theta(0, tau, 1)*exp(2*I*pi*u*xi)*theta(eta + tau*xi + u, tau, 1)/(theta(u, tau)*theta(eta + tau*xi, tau))
 -theta(eta + tau*xi + u, tau)*theta(0, tau, 1)*exp(2*I*pi*u*xi)*theta(u, tau,1)/(theta(u, tau)**2*theta(eta + tau*xi, tau))+
 2*I*pi*xi*theta(eta + tau*xi + u, tau)*theta(0, tau, 1)*exp(2*I*pi*u*xi)/(theta(u, tau)*theta(eta + tau*xi, tau))),
   -Sum(Sum(exp(2*I*pi*(-m*xi + n*eta))/(u + m + n*tau)**2,(n,-N,N)),(m,-M,M)))

Eq(2*I*pi*xi*theta(eta + tau*xi + u, tau)*theta(0, tau, 1)*exp(2*I*pi*u*xi)/(theta(u, tau)*theta(eta + tau*xi, tau)) + theta(0, tau, 1)*theta(eta + tau*xi + u, tau, 1)*exp(2*I*pi*u*xi)/(theta(u, tau)*theta(eta + tau*xi, tau)) - theta(eta + tau*xi + u, tau)*theta(0, tau, 1)*theta(u, tau, 1)*exp(2*I*pi*u*xi)/(theta(u, tau)**2*theta(eta + tau*xi, tau)), -Sum(exp(2*I*pi*(eta*n - m*xi))/(m + n*tau + u)**2, (n, -N, N), (m, -M, M)))

In [241]:
S_val_1_diff_u = Eq(Derivative(Seq_dbl_ktheta.lhs,u), theta_ratio_S_u_diff(xi_val, eta_val, u_val, tau_val))
S_val_2_diff_u = Eq(Derivative(Seq_dbl_series.lhs,u), dbl_series_S(xi_val, eta_val, u_val, tau_val, M_val, N_val, _s=2))
S_val_1_diff_u
S_val_2_diff_u
abs(S_val_1_diff_u.rhs - S_val_2_diff_u.rhs) / abs(S_val_1_diff_u.rhs + S_val_2_diff_u.rhs) / 2

Eq(Derivative(Ser(xi, eta, u, tau), u), 1.38332108621588 - 1.14741674471763*I)

Eq(Derivative(Ser(xi, eta, u, tau), u), 1.38415855663739 - 1.1473988546033*I)

0.000116498738477364

### Kronecker-Eisenstein Series
Charollois and Sczech go on to introduce the following Kronecker-Eisenstein series (where lattice points that make the denominator zero should be ommitted). Where they sum over lattice points denoted $\lambda$; I explicitly write those points as $n \tau +m$, with $\bar{\lambda}=m+n\bar{\tau}$.

Note: I think Charollois and Sczech have a typo in the start of section 3 in their definition of the following function, I think they meant to write $\psi(\lambda \bar{w})$ instead of $\psi(z \bar{w})$ in the sum else the character could be factored out.

In [92]:
Kr_Es_dbl_series_K = Eq(K(z,w,s,tau,a), 
   Sum(Sum(
       (conjugate(z) + m + n * conjugate(tau)) ** a / 
       abs(z + m + n * tau) ** (2*s) 
       * exp(((m + n * tau)*conjugate(w) - (m + n * conjugate(tau))*w) * pi/im(tau))
    ,(n,-N,N)),(m,-M,M))
  )
re(s)>a/2+1
Kr_Es_dbl_series_K

re(s) > a/2 + 1

Eq(K(z, w, s, tau, a), Sum((m + n*conjugate(tau) + conjugate(z))**a*exp(pi*(-w*(m + n*conjugate(tau)) + (m + n*tau)*conjugate(w))/im(tau))/Abs(m + n*tau + z)**(2*s), (n, -N, N), (m, -M, M)))

They then link this back to the double series developed earlier, namely $\mathrm{Ser}(\xi, \eta, u,\tau )$, by considering the following special case:

In [93]:
Kr_Es_dbl_series_K.subs([(a,1),(s,1)])
Eq(K(z, w, tau), Kr_Es_dbl_series_K.lhs.subs([(a,1),(s,1)]))
Eq(K(z, w, tau), Ser(xi, eta, z , tau))

Eq(K(z, w, 1, tau, 1), Sum((m + n*conjugate(tau) + conjugate(z))*exp(pi*(-w*(m + n*conjugate(tau)) + (m + n*tau)*conjugate(w))/im(tau))/Abs(m + n*tau + z)**2, (n, -N, N), (m, -M, M)))

Eq(K(z, w, tau), K(z, w, 1, tau, 1))

Eq(K(z, w, tau), Ser(xi, eta, z, tau))

In [94]:
Kr_Es_dbl_series_K_a1_s1 = Eq(
    K(z, w, tau), 
    Sum(Sum(
        exp(((m + n * tau)*conjugate(w) - (m + n * conjugate(tau))*w) * pi/im(tau)) / (z + m + n * tau) 
    ,(n,-N,N)),(m,-M,M))
)
Kr_Es_dbl_series_K_a1_s1

Eq(K(z, w, tau), Sum(exp(pi*(-w*(m + n*conjugate(tau)) + (m + n*tau)*conjugate(w))/im(tau))/(m + n*tau + z), (n, -N, N), (m, -M, M)))

Where $z, w \in \mathbb{C} $ (but they say $z$ cannot be a lattice point), and where $w$ is related to $\eta, \xi, \tau$ in the following way:

In [95]:
nm_character_arg_w_tau = (((m + n * tau)*conjugate(w) - (m + n * conjugate(tau))*w) * pi/im(tau))
w_tau_eta_xi = Eq(w,eta+xi*tau)
w_tau_eta_xi_conj = Eq(conjugate(w), eta +xi*conjugate(tau))
nm_character_arg_w_tau_eta_xi = Eq(
    nm_character_arg_w_tau,
    nm_character_arg_w_tau
    .subs([w_tau_eta_xi_conj.args, w_tau_eta_xi.args])
    .subs([(tau, re(tau) + I *im(tau))])
    .expand().factor()
)
eta_xi_sols_w_tau = solve([w_tau_eta_xi, w_tau_eta_xi_conj],[xi, eta])
eta_tau_w = Eq(eta, eta_xi_sols_w_tau[eta].simplify())
xi_tau_w = Eq(xi, eta_xi_sols_w_tau[xi].simplify())

Eq(im(xi),0)
Eq(im(eta),0)
w_tau_eta_xi
w_tau_eta_xi_conj
nm_character_arg_w_tau_eta_xi
eta_tau_w
xi_tau_w

Eq(im(xi), 0)

Eq(im(eta), 0)

Eq(w, eta + tau*xi)

Eq(conjugate(w), eta + xi*conjugate(tau))

Eq(pi*(-w*(m + n*conjugate(tau)) + (m + n*tau)*conjugate(w))/im(tau), -2*I*pi*(-eta*n + m*xi))

Eq(eta, (tau*conjugate(w) - w*conjugate(tau))/(tau - conjugate(tau)))

Eq(xi, (w - conjugate(w))/(tau - conjugate(tau)))

They then introduce a new function $\Theta$ and it is this function that they refer to as the *Kronecker theta function*. It is deifned as:

In [96]:
Eq(Theta(z, w, tau), K(z, w, tau)*exp(-z*conjugate(w)*pi/im(tau)))
Eq(Theta(z, w, tau), exp(-z*conjugate(w)*pi/im(tau))*Sum(Sum(
        exp(((m + n * tau)*conjugate(w) - (m + n * conjugate(tau))*w) * pi/im(tau)) / (z + m + n * tau) 
    ,(n,-N,N)),(m,-M,M)))

Eq(Theta(z, w, tau), K(z, w, tau)*exp(-pi*z*conjugate(w)/im(tau)))

Eq(Theta(z, w, tau), exp(-pi*z*conjugate(w)/im(tau))*Sum(exp(pi*(-w*(m + n*conjugate(tau)) + (m + n*tau)*conjugate(w))/im(tau))/(m + n*tau + z), (n, -N, N), (m, -M, M)))

In [97]:
Eq(Seq_dbl_ktheta.lhs.subs(u,z), Seq_dbl_ktheta.rhs.subs([(u,z), (eta + tau*xi, w), xi_tau_w.subs(tau-conjugate(tau),2*I*im(tau)).args]))

Eq(Ser(xi, eta, z, tau), theta(w + z, tau)*theta(0, tau, 1)*exp(pi*z*(w - conjugate(w))/im(tau))/(theta(w, tau)*theta(z, tau)))

In [98]:
exp(2*I*pi*xi*u)*theta(0,tau,1)*theta(u + eta + tau*xi, tau)/theta(u, tau)/theta(eta + tau*xi, tau)

theta(eta + tau*xi + u, tau)*theta(0, tau, 1)*exp(2*I*pi*u*xi)/(theta(u, tau)*theta(eta + tau*xi, tau))

In [99]:
eta_xi_sols_w_tau = solve([w_tau_eta_xi, w_tau_eta_xi_conj],[xi, eta])
eta_tau_w = Eq(eta, eta_xi_sols_w_tau[eta].simplify())
xi_tau_w = Eq(xi, eta_xi_sols_w_tau[xi].simplify())

In [101]:
kf1 = ktheta_dbl_series.subs([xi_tau_w.subs(tau-conjugate(tau),2*I*im(tau)).args, eta_tau_w.subs(tau-conjugate(tau),2*I*im(tau)).args])

In [109]:
Eq((kf1.lhs.subs(conjugate(tau), tau-2*I*im(tau)).simplify()*exp(-pi*u*(w-conjugate(w))/im(tau))).simplify(), 
   exp(-pi*u*(w-conjugate(w))/im(tau))*kf1.rhs)

Eq(theta(u + w, tau)*theta(0, tau, 1)/(theta(u, tau)*theta(w, tau)), exp(-pi*u*(w - conjugate(w))/im(tau))*Sum(exp(2*I*pi*(I*m*(w - conjugate(w))/(2*im(tau)) - I*n*(tau*conjugate(w) - w*conjugate(tau))/(2*im(tau))))/(m + n*tau + u), (n, -N, N), (m, -M, M)))

In [90]:
exp(((n + m * tau)*conjugate(w) - (n + m * conjugate(tau))*w) * pi/im(tau)).expand()

exp(-pi*n*w/im(tau))*exp(pi*n*conjugate(w)/im(tau))*exp(pi*m*tau*conjugate(w)/im(tau))*exp(-pi*m*w*conjugate(tau)/im(tau))